In [1]:
import os
import json
from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_absolute_error
from scipy.sparse import csr_matrix, hstack
from sklearn.linear_model import Ridge

In [2]:
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [3]:
def read_json_line(line=None):
    result = None
    try:        
        result = json.loads(line)
    except Exception as e:      
        # Find the offending character index:
        idx_to_replace = int(str(e).split(' ')[-1].replace(')',''))      
        # Remove the offending character:
        new_line = list(line)
        new_line[idx_to_replace] = ' '
        new_line = ''.join(new_line)     
        return read_json_line(line=new_line)
    return result

In [7]:
def extract_features_and_write(path_to_data,
                               inp_filename, is_train=True):
    
    features = ['content', 'published', 'title', 'author']
    prefix = 'train' if is_train else 'test'
    feature_files = [open(os.path.join(path_to_data,
                                       '{}_{}.txt'.format(prefix, feat)),
                          'w', encoding='utf-8')
                     for feat in features]
    
    with open(os.path.join(path_to_data, inp_filename), 
              encoding='utf-8') as inp_json_file:

        for line in tqdm_notebook(inp_json_file):
            json_data = read_json_line(line)
            json_data = read_json_line(line)
            [feature_files[i].write(json_data[features[i]]) for i in range(len(feature_files))]

In [5]:
PATH_TO_DATA = 'C:\mlcourse\Databases\kaggle_medium'

In [8]:
extract_features_and_write(PATH_TO_DATA, 'train.json', is_train=True)

TypeError: write() argument must be str, not dict

In [ ]:
extract_features_and_write(PATH_TO_DATA, 'test.json', is_train=False)

In [ ]:
### to test functions

features = ['content', 'published', 'title', 'author']
prefix = 'train1' 
feature_files = [open(os.path.join(PATH_TO_DATA, 
                                   '{}_{}.txt'.format(prefix, feat)), 'w', encoding='utf-8')
                 for feat in features]


[feature_files[i].write(features[i] + ";/n") for i in range(len(feature_files))]

    
for i in feature_files:
    i.close

